In [1]:
import pandas as pd
from datetime import date

In [2]:
REINGRESADOS = [
    2222985,
    2220048,
    2220040
]
CANCELADOS = pd.read_excel('./INPUT/DEFINITIVAS/CANCELADOS ANTERIORES.xlsx', engine = 'openpyxl')['CODIGO_UIS'].unique()
NIV_BIN = pd.read_excel('./INPUT/PROGRAMA2/NivelacionBinario_2022-12-01.xlsx', engine = 'openpyxl')
REPORTES_NIV = [
    'INFORME_NIVELACIÓN_4A2_2022-12-01_Hora-7.xlsx',
    'INFORME_NIVELACIÓN_4B2_2022-12-01_Hora-7.xlsx'
]
ANIO = 2022
HOJA_APROBADOS = 'Estudiantes que aprobaron'
COL_NOTA = 'RETO_1'
CURSO_NIVS = 'NIV'
CICLOS = {
    #LLAVE: COD_ASIG, N_EVAS, PERIODO
    'NIV_CICLO_1':(2411, 5, 1),
    'NIV_CICLO_2':(2511, 5, 2),
    'NIV_CICLO_3':(2611, 4, 3),
    'NIV_CICLO_4A':(2711, 4, 4),
    'NIV_CICLO_4B':(2714, 4, 4),
}

In [3]:
# Consolidado aprobados
dfReportes = []
for reporte in REPORTES_NIV:
    dfReportes.append(pd.read_excel('./INPUT/PROGRAMA2/' + reporte, engine = 'openpyxl', sheet_name=HOJA_APROBADOS))
consolidado = pd.concat(dfReportes)

In [4]:
NIV_BIN.head()

,CÓDIGO,NOMBRE,APELLIDO,RAMA,NIV_CICLO_1,NIV_CICLO_2,NIV_CICLO_3,NIV_CICLO_4
0,2223526,ALVAREZ BARAJAS,ANGEL DANILO,4A,False,False,True,True
1,2221267,ARGUMEDO TORRES,NAYIB ANTONIO,4A,False,False,True,True
2,2223444,AVILA OLAYA,JONATHAN DAVID,4B,False,False,False,False
3,2224083,AVILA RODRIGUEZ,SERGIO ANDRES,4A,True,True,True,True
4,2222304,CANON,JUAN ESTEBAN,4A,False,False,False,False


In [5]:
# Discriminación por ciclo
aprobados = consolidado['Código UIS'].unique()
aprobadosCiclos = {}
for ciclo in CICLOS:
    aprobadosCiclos[ciclo] = []
for index, row in NIV_BIN.iterrows():
    # Descartar cancelados
    reingreso = row['CÓDIGO'] in REINGRESADOS
    if row['CÓDIGO'] in CANCELADOS and not (reingreso):
        continue
    # Descartar reprobados o innecesarios
    if row['CÓDIGO'] not in aprobados:
        continue
    for i in range(1, 5):
        nCol = 'NIV_CICLO_' + str(i)
        # ¿Debe nivelar ese ciclo?
        if row[nCol]:
            # Si la columna pertenece al ciclo 4, añade el sufijo del A o B
            nDict = nCol if i != 4 else nCol + row['RAMA'][-1]
            # Añade tuplas código, nota
            nota = consolidado[consolidado['Código UIS'] == row['CÓDIGO']][COL_NOTA].iloc[0]
            aprobadosCiclos[nDict].append((row['CÓDIGO'], nota))

In [6]:
for llave, valor in CICLOS.items():
    print (llave, valor)

NIV_CICLO_1 (2411, 5, 1)
NIV_CICLO_2 (2511, 5, 2)
NIV_CICLO_3 (2611, 4, 3)
NIV_CICLO_4A (2711, 4, 4)
NIV_CICLO_4B (2714, 4, 4)


In [7]:
# Generación de archivos
for llave, valor in CICLOS.items():
    csvParcial = {
        'CODIGO ESTUDIANTE': [],
        'AÑO': [],
        'PERIODO': [],
        'CODIGO ASIGNATURA': [],
        'NUMERO PARCIAL': [],
        'GRUPO': [],
        'NOTA': []
    }
    csvDef = {
        'CODIGO ESTUDIANTE': [],
        'AÑO': [],
        'PERIODO': [],
        'CODIGO ASIGNATURA': [],
        'GRUPO': [],
        'NOTA DEFINITIVA': []
    }
    for cod, nota in aprobadosCiclos[llave]:
        csvDef['CODIGO ESTUDIANTE'].append(cod)
        csvDef['AÑO'].append(ANIO)
        csvDef['PERIODO'].append(valor[2])
        csvDef['CODIGO ASIGNATURA'].append(valor[0])
        csvDef['GRUPO'].append(CURSO_NIVS)
        csvDef['NOTA DEFINITIVA'].append(nota)
        for i in range (1, valor[1] + 1):
            csvParcial['CODIGO ESTUDIANTE'].append(cod)
            csvParcial['AÑO'].append(ANIO)
            csvParcial['PERIODO'].append(valor[2])
            csvParcial['CODIGO ASIGNATURA'].append(valor[0])
            csvParcial['NUMERO PARCIAL'].append(i)
            csvParcial['GRUPO'].append(CURSO_NIVS)
            csvParcial['NOTA'].append(nota)
    pd.DataFrame(csvDef).to_csv(f'./OUTPUT/CSVS/PROGRAMA2/NOTAS_DEFINITIVAS_{llave}_{valor[0]}_' + str(date.today()) + '.csv', sep=';', encoding = 'utf-8', index=False)
    pd.DataFrame(csvParcial).to_csv(f'./OUTPUT/CSVS/PROGRAMA2/NOTAS_PARCIALES_{llave}_{valor[0]}_' + str(date.today()) + '.csv', sep=';', encoding = 'utf-8', index=False)